In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/Users/devansh/Desktop/bullyin tweets/cyberbullying_tweets.csv")

In [3]:
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [4]:
df.isnull().sum()

tweet_text            0
cyberbullying_type    0
dtype: int64

In [5]:
df.dropna(inplace=True)
df.shape

(47692, 2)

In [6]:
df['cyberbullying_type'] = '__label__' + df['cyberbullying_type'].astype(str)
df.head(5)

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",__label__not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,__label__not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,__label__not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",__label__not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,__label__not_cyberbullying


In [7]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

def preprocess_text(text):
    # Remove hashtags
    text = text.replace('#', '')
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    stop_words.update(list(string.punctuation))
    filtered_tokens = [token.lower() for token in tokens if token.lower() not in stop_words]
    
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    
    # Join the lemmatized tokens back into a string
    processed_text = ' '.join(lemmatized_tokens)
    
    return processed_text
df['clean_text'] = df['tweet_text'].apply(preprocess_text)


[nltk_data] Downloading package punkt to /Users/devansh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/devansh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
df

,tweet_text,cyberbullying_type,clean_text
0,"In other words #katandandre, your food was cra...",__label__not_cyberbullying,word katandandre food crapilicious mkr
1,Why is #aussietv so white? #MKR #theblock #ImA...,__label__not_cyberbullying,aussietv white mkr theblock imacelebrityau tod...
2,@XochitlSuckkks a classy whore? Or more red ve...,__label__not_cyberbullying,xochitlsuckkks classy whore red velvet cupcake
3,"@Jason_Gio meh. :P thanks for the heads up, b...",__label__not_cyberbullying,jason_gio meh p thanks head concerned another ...
4,@RudhoeEnglish This is an ISIS account pretend...,__label__not_cyberbullying,rudhoeenglish isi account pretending kurdish a...
...,...,...,...
47687,"Black ppl aren't expected to do anything, depe...",__label__ethnicity,black ppl n't expected anything depended anyth...
47688,Turner did not withhold his disappointment. Tu...,__label__ethnicity,turner withhold disappointment turner called c...
47689,I swear to God. This dumb nigger bitch. I have...,__label__ethnicity,swear god dumb nigger bitch got bleach hair re...
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,__label__ethnicity,yea fuck rt therealexel youre nigger fucking u...


In [9]:
df['category_description'] = df['cyberbullying_type'] + ' ' + df['clean_text']
df.head(3)

,tweet_text,cyberbullying_type,clean_text,category_description
0,"In other words #katandandre, your food was cra...",__label__not_cyberbullying,word katandandre food crapilicious mkr,__label__not_cyberbullying word katandandre fo...
1,Why is #aussietv so white? #MKR #theblock #ImA...,__label__not_cyberbullying,aussietv white mkr theblock imacelebrityau tod...,__label__not_cyberbullying aussietv white mkr ...
2,@XochitlSuckkks a classy whore? Or more red ve...,__label__not_cyberbullying,xochitlsuckkks classy whore red velvet cupcake,__label__not_cyberbullying xochitlsuckkks clas...


In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [11]:
import fasttext

model = fasttext.train_supervised(input="cyberbullying.train")
model.test("cyberbullying.test")

Read 0M words
Number of words:  30778
Number of labels: 6
Progress: 100.0% words/sec/thread: 1877086 lr:  0.000000 avg.loss:  0.472231 ETA:   0h 0m 0s


(9539, 0.8130831324038159, 0.8130831324038159)

In [12]:
import fasttext
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'], df['cyberbullying_type'], test_size=0.2, random_state=42)


In [14]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

model = fasttext.train_supervised(input='cyberbullying.train')

Read 0M words
Number of words:  30778
Number of labels: 6
Progress: 100.0% words/sec/thread: 1959957 lr:  0.000000 avg.loss:  0.442005 ETA:   0h 0m 0s


In [16]:
X_train_ft = np.array([model.get_sentence_vector(text) for text in X_train])
X_test_ft = np.array([model.get_sentence_vector(text) for text in X_test])

In [17]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_ft, y_train_enc)
y_pred_rf = rf.predict(X_test_ft)

In [18]:
lr = LogisticRegression(random_state=42)
lr.fit(X_train_ft, y_train_enc)
y_pred_lr = lr.predict(X_test_ft)

/Users/devansh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print('RF accuracy:', accuracy_score(y_test_enc, y_pred_rf))
print('RF precision:', precision_score(y_test_enc, y_pred_rf, average='macro'))
print('RF recall:', recall_score(y_test_enc, y_pred_rf, average='macro'))
print('RF f1-score:', f1_score(y_test_enc, y_pred_rf, average='macro'))
print('LR accuracy:', accuracy_score(y_test_enc, y_pred_lr))
print('LR precision:', precision_score(y_test_enc, y_pred_lr, average='macro'))
print('LR recall:', recall_score(y_test_enc, y_pred_lr, average='macro'))
print('LR f1-score:', f1_score(y_test_enc, y_pred_lr, average='macro'))

RF accuracy: 0.8486214487891812
RF precision: 0.8507148998707607
RF recall: 0.8499628657723001
RF f1-score: 0.8500913186699347
LR accuracy: 0.8768214697557396
LR precision: 0.8789125567830345
LR recall: 0.877742447766176
LR f1-score: 0.8776554728886367


NameError: name 'X' is not defined

In [22]:
import os

os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

In [ ]:

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout

from tensorflow.keras.utils import plot_model

#Library for evaluation
from sklearn import metrics
from functools import reduce
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
embed_dim = 32
lstm_out = 32
model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(Dropout(0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.4))
model.add(Dense(1,activation='sigmoid'))
adam = optimizers.Adam(learning_rate=0.002)
model.compile(loss = 'binary_crossentropy', optimizer=adam ,metrics = ['accuracy'])
print(model.summary())

In [ ]:
model.fit(X_train, y_train, epochs = 10, batch_size=32, validation_data=(X_test, y_test))